# Predict whether wine is "good" or "bad"
## Read the data
### Load libraries

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

### Load data

In [31]:
wine = pd.read_csv('data/wine_clean.csv')
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,density,pH,sulphates,alcohol,quality_bin,type_n
0,7.0,0.27,0.36,20.7,0.045,45.0,1.0010,3.00,0.45,8.8,1,1
1,6.3,0.30,0.34,1.6,0.049,14.0,0.9940,3.30,0.49,9.5,1,1
2,8.1,0.28,0.40,6.9,0.050,30.0,0.9951,3.26,0.44,10.1,1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,0.9956,3.19,0.40,9.9,1,1
4,7.2,0.23,0.32,8.5,0.058,47.0,0.9956,3.19,0.40,9.9,1,1


Quality_bin is 1 if original quality score was greater than 5
Check the datatypes on the columns

In [32]:
wine.info()

<class 'pandas.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fixed acidity        6487 non-null   float64
 1   volatile acidity     6489 non-null   float64
 2   citric acid          6494 non-null   float64
 3   residual sugar       6495 non-null   float64
 4   chlorides            6495 non-null   float64
 5   free sulfur dioxide  6497 non-null   float64
 6   density              6497 non-null   float64
 7   pH                   6488 non-null   float64
 8   sulphates            6493 non-null   float64
 9   alcohol              6497 non-null   float64
 10  quality_bin          6497 non-null   int64  
 11  type_n               6497 non-null   int64  
dtypes: float64(10), int64(2)
memory usage: 609.2 KB


All columns are numeric and ready to be modeled. Nulls will need to be imputed in the cross validation

## Grid Search CV

In the exploratory analysis about 30% of rows appeared to be 0 for quality_bin. Classes do not need to be more balanced

### Train Test Split

In [33]:
X = wine.drop(columns=['quality_bin'])
y = wine['quality_bin']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size= 0.2, random_state = 40)

### Set up Grid Search

In [34]:
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
])

param_grid = [
    {
        'scaler': [StandardScaler()],
        'model': [LogisticRegression(max_iter = 5000)],
        'model__C': [0.1, 1, 10]
    },
    {
        'scaler': [StandardScaler()],
        'model': [SVC()],
        'model__C': [0.1, 1, 10],
        'model__kernel': ['linear', 'rbf']
    },
    {
        'scaler': ['passthrough'],
        'model': [XGBClassifier(eval_metric='logloss')],
        'model__n_estimators': [100, 200],
        'model__max_depth': [3, 5]
    }
]

In [35]:
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

In [36]:
grid = GridSearchCV(
    pipe,
    param_grid,
    cv=cv,
    scoring= 'roc_auc',
    n_jobs=-1
)

grid.fit(X_train, y_train)


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...egression())])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","[{'model': [LogisticRegre...max_iter=5000)], 'model__C': [0.1, 1, ...], 'scaler': [StandardScaler()]}, {'model': [SVC()], 'model__C': [0.1, 1, ...], 'model__kernel': ['linear', 'rbf'], 'scaler': [StandardScaler()]}, ...]"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo... shuffle=True)


The winning model is the XGBClassifier

In [37]:
print(grid.best_estimator_)
print(grid.best_params_)
print(grid.best_score_)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', 'passthrough'),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='logloss',
                               feature_types=None, feature_weights=None,
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan

## Test on Test Set

In [38]:
y_pred_proba = grid.predict_proba(X_test)[:, 1]

test_auc = roc_auc_score(y_test, y_pred_proba)

print("Test ROC-AUC:", test_auc)

Test ROC-AUC: 0.8835650111699539


This model has high predictive power!